# Installation de Ollama

Dans un terminal linux lancer la commande suivante : 

```sh
curl -fsSL https://ollama.com/install.sh | sh
```

### Utilisation d'un modèle

Utilisation du modèle llama3.

Si celui-ci n'est pas encore présent en local, alors ollama le télécharge au préalable avant de s'éxécuter.

```sh
ollama run llama3
```

In [1]:
import ollama
from ollama import Client
import uuid
import json

client = Client(host='http://localhost:11434')

# print(ollama.invoke("quelle était ma première question ?"))


In [2]:
def graphPrompt(input: str, metadata={}, model="llama3:latest"):
        
    chunk_id = metadata.get('chunk_id', uuid.uuid4().hex)

    # model_info = client.show(model_name=model)
    # print( chalk.blue(model_info))

    SYS_PROMPT = ("You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include person (agent), location, organization, date, duration, \n"
            "\tcondition, concept, object, entity  etc.\n"
            "\tTerms should be as atomistic\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms."
        "Never add a comment"
        "and the relation between them like the follwing. NEVER change the value of the chunk_ID as defined in this prompt: \n"
        "[\n"
        "   {\n"
        #'       "chunk_id": "CHUNK_ID_GOES_HERE",\n'
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or four words max"\n' 
        "   }, {...}\n"
        "]"
    )
    # SYS_PROMPT = (
    #     "tu vas avoir pour tâche de faire une analyse onthologique du texte dans le but de générer un graphe de connaissance."        
    # )
    SYS_PROMPT = SYS_PROMPT.replace('CHUNK_ID_GOES_HERE', chunk_id)

    USER_PROMPT = f"context: ```{input}``` \n\n output: "

    response = client.generate(model, system=SYS_PROMPT, prompt=USER_PROMPT)
    
    aux1 = response['response']
    print(aux1)
    # Find the index of the first open bracket '['
    start_index = aux1.find('[')
    end_index = aux1.find(']')
    # Slice the string from start_index to extract the JSON part and fix an unexpected problem with insertes escapes (WHY ?)
    json_string = aux1[start_index:end_index+1]
    # json_string = json_string.replace('\\\\\_', '_')
    # json_string = json_string.replace('\\\\_', '_')
    # json_string = json_string.replace('\\\_', '_')
    # json_string = json_string.replace('\\_', '_')
    # json_string = json_string.replace('\_', '_')
    json_string = json_string.replace('```', '')
    json_string = json_string.replace("\n", '')
    json_string = json_string.replace("\t", '')
    json_string = json_string.replace("  ", '')
    json_string.lstrip() # eliminate eventual leading blank spaces
    #####################################################
    # print("json-string:\n" + json_string)
    #####################################################         
    # try:
    #     result = json.loads(json_string)
    #     result = [dict(item) for item in result]
    # except Exception as error:
    #     print("error ici : ", error)
    #     print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
    #     result = None
    # # print("§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§")

    # return result
    return aux1

def resume(input: str, metadata={}, model="llama3:latest"):
    SYS_PROMPT = (
        "tu dois fusionner le contexte afin d'en créer un arbre de connaissance."
        "Toutes les entités que tu trouves sont des noeuds"
        "Une entité peut réprésenter un fruit, un arbre, un lieu, une couleur, une condition, une possibilité, un fait"
        "Chaque noeud et lien sont représentés par des termes atomiques de 3 mots max"
        "Le texte est délimité par les caractères ```"
        "La réponse dois être en français."
        "Le format de sortie est un json comme défini dans ce prompt :"        
        '[{ "node_1" : "...", "node_2" : "...", "edge" : "lien qui lie le node_1 et le node_2" }, {...}]'
        "Un noeud doit avoir absolument avec un lien caractérisé par l'attribut 'edge' dans la sortie json"
        "avant de rendre ta solution tu vérifies l'orthographe des termeset tu les traduis en français"
    )

    USER_PROMPT = f"context: ```{input}``` \n\n output: "

    response = client.generate(model, system=SYS_PROMPT, prompt=USER_PROMPT)
    
    return response['response']

In [3]:
sall = []
for i in range(3):
    print('test', i)
    res = graphPrompt("La pomme est verte. Elle pousse dans un arbre. Cet arbre s'appelle un pommier. Sachant que sa couleur est verte, est-il possible d'avoir une pomme rouge ?")
    sall.append(res)

print("RESUMER : ", resume("\n".join(sall)))

test 0
Here is the extracted ontology and relations:

[
    {
        "node_1": "pomme",
        "node_2": "arbre",
        "edge": "grows"
    },
    {
        "node_1": "pommier",
        "node_2": "arbre",
        "edge": "is type of"
    },
    {
        "node_1": "pomme",
        "node_2": "verte",
        "edge": "has color"
    },
    {
        "node_1": "pommier",
        "node_2": "pomme",
        "edge": "produces"
    },
    {
        "node_1": "pomme",
        "node_2": "rouge",
        "edge": "can be in color"
    }
]

Let me know if you'd like me to explain the reasoning behind each extracted term and relation!
test 1
[ 
    { "node_1": "pomme", "node_2": "arbre", "edge": "grows in" }, 
    { "node_1": "pommier", "node_2": "arbre", "edge": "is a type of" }, 
    { "node_1": "pomme", "node_2": "verte", "edge": "has color" }, 
    { "node_1": "arbre", "node_2": "pommier", "edge": "hosts" }, 
    { "node_1": "pomme", "node_2": "rouge", "edge": "can be colored with" } 
]
tes